In [4]:
import numpy as np
import pandas as pd
import os
import re

In [5]:
sharks = pd.read_csv("C:/PATRICIA/Ironhack/Projects/PROJECT-1/data/attacks.csv",encoding= 'unicode_escape')

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
sharks 

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
sharks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [50]:
#as columns 'Unnamed: 22' (=25722 Nan )'Unnamed: 23'(=25721 Nan) and over 25723 rows of sharks => no sense to keep them
sharks1=sharks.drop(columns = ['Unnamed: 22', 'Unnamed: 23'])

In [51]:
sharks1.shape

(25723, 22)

In [52]:
# eliminar todas las filas que tengan nulos todos sus valores (axis = 0 xq son filas y how ='all' xq sólo quiero que me
#elimine las que tengan todos los datos NaN)
sharks2=sharks1.dropna(
    axis=0,
    how='all')

In [53]:
sharks2.shape

(8703, 22)

In [54]:
#eliminando filas duplicadas
sharks3 = sharks2.drop_duplicates()

In [55]:
sharks3.shape

(6311, 22)

In [56]:
#verifico cuántos valores divergen entre las columnas Case Number'y'Case Number.1 
subset_case = sharks3[sharks3['Case Number'] != sharks3['Case Number.1']]
len(subset_case[['Case Number','Case Number.1']])

33

In [57]:
subset_case = sharks3[sharks3['Case Number'] != sharks3['Case Number.2']]
len(subset_case[['Case Number','Case Number.2']])

13

In [58]:
#a la vista que la diferencia es mínima y que, visualmente se trata de una diferencia de 1 día, puedo prescindir de estas columans
sharks4=sharks3.drop(columns = ['Case Number.1', 'Case Number.2'])

In [59]:
# extraems día, mes y año de la columna 'case number' ya que el código se genera a través de esta info
sharks4[['year_c', 'month_c', 'day_c']] = sharks4['Case Number'].str.extract(r'(\d{4})\.(\d{2})\.(\d{2})')

In [60]:
sharks4.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,year_c,month_c,day_c
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0,2018,06,25
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0,2018,06,18
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0,2018,06,09
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0,2018,06,08
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0,2018,06,04


In [61]:
#verificando diferencias entre el año de 'Date' y el extraido de 'Case Number'
sharks4['Year'] = sharks4['Year'].astype(str)
year_check = sharks4[sharks4['Year'].str.lower().str[:4] != sharks4['year_c'].str.lower()]
year_check.drop_duplicates()
a = len(year_check)
b = len(year_check['Year'] == 'nan')
print (f"los valores no repetidios entre year y year_c son {a} de los cuales {b} son Nan")

los valores no repetidios entre year y year_c son 152 de los cuales 152 son Nan


In [62]:
#verificando diferencias entre el día de 'Date' y el extraido de 'Case Number'
sharks4['Date'] = sharks4['Date'].astype(str)
date_check = sharks4[sharks4['Date'].str.lower().str[:2] != sharks4['day_c'].str.lower()]
date_check.drop_duplicates()
a = len(date_check)
b = len(date_check['Date']=='nan')
print (f"los valores no repetidios entre el día indicado en Date y day_c son {a} de los cuales {b} son Nan")
date_check[['Date', 'day_c']].head(20)

los valores no repetidios entre el día indicado en Date y day_c son 1609 de los cuales 1609 son Nan


,Date,day_c
14,May 2018,00
17,Reported 30-Apr-2018,30
31,Reported 10-Apr-2018,10
37,9-Mar-2018,09
38,9-Mar-2018,09
41,18-Feb-2018,17
59,Reported 25-Nov-2017,25
62,Reported 13-Nov-2017,13
65,Reported 31-Oct-2017,31
86,Sep-2017,14


In [63]:
# tras las comprobacions anteriores parece correcto eliminar Case Number, Date, Year y usar los extraidos de 'Case Number'
sharks5=sharks4.drop(columns = ['Case Number', 'Date' , 'Year'])

In [64]:
sharks4.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,year_c,month_c,day_c
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0,2018,06,25
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0,2018,06,18
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0,2018,06,09
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0,2018,06,08
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0,2018,06,04


In [65]:
sharks5['href'][0]

'http://sharkattackfile.net/spreadsheets/pdf_directory/2018.06.25-Wolfe.pdf'

In [66]:
#verificado que los pdf, href formula y href tampoco aportan  nada elimino estas columnas, 
#igual que las otras de case no y originalorder
sharks6=sharks5.drop(columns = ['href', 'pdf' , 'href formula', 'original order'])

In [67]:
sharks6.head()

,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,year_c,month_c,day_c
0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018,06,25
1,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018,06,18
2,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018,06,09
3,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018,06,08
4,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018,06,04


In [68]:
sharks6.shape

(6311, 16)

In [69]:
sharks6.isna().sum()

Type                        13
Country                     59
Area                       464
Location                   549
Activity                   553
Name                       219
Sex                        574
Age                       2840
Injury                      37
Fatal (Y/N)                548
Time                      3363
Species                   2847
Investigator or Source      26
year_c                     139
month_c                    139
day_c                      139
dtype: int64

In [70]:
set(sharks6['Activity'])

{'Diving from the lugger San, operated by the Protector of the Aborigines',
 'Diving in aquarium display tank',
 'Wading & pushing dinghy toward  the shallows',
 'Canoe capsized by shark',
 'Fishing, holding fish',
 'Tandem surfing',
 'Skindiving for specimens',
 'Diving / UW photography',
 "Boat capsized between O'ahu & Molokai",
 'Wading after stray fish from seine netter\x92s catch',
 'Paddleskiing',
 "Zosimo & his son, Jeffrey Popa, failed to return from overnight fishing trip in a 14' boat, Boat apparently sank, debris recovered but his son & boat were never found",
 'Fishing, hauling in net, shark in net',
 'Sea disaster',
 'Fishing (big game)',
 'Hunting sharks',
 "40' fishing boat sank",
 'Adrift on refugee raft',
 'Swimming with fish attached to  belt',
 'Wading / fishing & carrying a bag of fish',
 'Surf fishing / wading',
 'Surf fishing in waist-deep water',
 'Swimming with dog near canning factory',
 'Fell overboard from the steamship Chala',
 'Boating',
 'Spear fishing',
 

In [71]:
sharks6.columns

Index(['Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ',
       'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ',
       'Investigator or Source', 'year_c', 'month_c', 'day_c'],
      dtype='object')

In [72]:
#para poder intentar clasificar el tipo de ataque tengo que eliminar los Nan de esa columna primero:
sharks6_activity = sharks5.dropna(subset =['Activity'])

In [73]:
sharks6_activity.shape

(5758, 20)

In [74]:
sharks6_activity.isna().sum() # verificamos que en Activity no quedan NaN

Type                         4
Country                     43
Area                       376
Location                   453
Activity                     0
Name                       151
Sex                        414
Age                       2429
Injury                      16
Fatal (Y/N)                414
Time                      2876
Species                   2523
Investigator or Source      13
pdf                          0
href formula                 1
href                         0
original order               0
year_c                     111
month_c                    111
day_c                      111
dtype: int64

In [75]:
def activity(input_str):
    if re.search(r'\bSwimming\b', input_str, re.IGNORECASE):
        return 'swimming'
    if re.search(r'\bfishing\b', input_str, re.IGNORECASE):
        return 'fishing'
    if re.search(r'\bsurfing\b', input_str, re.IGNORECASE):
        return 'surfing'
    if re.search(r'\bboat\b', input_str, re.IGNORECASE):
        return 'sailing'
    if re.search(r'\bshark\b', input_str, re.IGNORECASE):
        return 'shark interacting'
    if re.search(r'\bdiving\b', input_str, re.IGNORECASE):
        return 'diving'
    return 'other'

In [76]:
sharks6_activity['Act_class'] = sharks6_activity['Activity'].apply(activity)

C:\Users\psaez\AppData\Local\Temp\ipykernel_19596\3992414460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks6_activity['Act_class'] = sharks6_activity['Activity'].apply(activity)


In [77]:
sharks6_activity.head(100)

,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,year_c,month_c,day_c,Act_class
0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0,2018,06,25,other
1,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0,2018,06,18,other
2,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0,2018,06,09,surfing
3,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0,2018,06,08,surfing
4,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0,2018,06,04,diving
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Unprovoked,AUSTRALIA,Victoria,Cathedral Rock,Surfing,Marcel Brundler,M,37,"No injury, board bitten",N,10h30,"White shark, 3 m","B. Myatt, GSAF",2017.08.27-Brundler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6207.0,2017,08,29,surfing
97,Unprovoked,USA,Florida,Bathtub Beach,Wading,Violet Veatch,F,3,Leg injured,N,13h19,NaN,"Sun Sentinel, 8/27/2017",2017.08.27-Veatch.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6206.0,2017,08,27,other
98,Invalid,SPAIN,Castellón,Grao de Moncofa,Swimming,female,F,11,Lacerations to left foot,NaN,Midday,Shark involvement questionable,"El Periodico Mediterraneo, 8/27/2017",2017.08.26.b-Spain.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6205.0,2017,08,26,swimming
99,Unprovoked,NEW ZEALAND,North Island,Off Tutukaka,Canoeing,Matt Kensington,M,NaN,"No injury, shark bit canoe",N,NaN,NaN,"C.Black, GSAF",2017.08.26.a-Kensington.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6204.0,2017,08,26,other


In [78]:
sharks6_activity['Act_class'].value_counts()

Act_class
other                2081
swimming             1117
surfing              1096
fishing               730
diving                532
shark interacting     125
sailing                77
Name: count, dtype: int64

In [79]:
sharks6_activity.shape

(5758, 21)

In [80]:
sharks6_activity.columns

Index(['Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex ',
       'Age', 'Injury', 'Fatal (Y/N)', 'Time', 'Species ',
       'Investigator or Source', 'pdf', 'href formula', 'href',
       'original order', 'year_c', 'month_c', 'day_c', 'Act_class'],
      dtype='object')

In [81]:
sharks6_activity.sample()

,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,year_c,month_c,day_c,Act_class
3519,Unprovoked,BAHAMAS,Out Islands,Andros Island,Photographing sharks underwater using Scuba,Richard Winer,M,41,"No injury, shark struck camera",N,Early afternoon,1.5 m to 1.8 m [5' to 6'] sandbar shark,"H.D. Baldridge, p.185",1967.08.00.b-Winer.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2784.0,1967,08,00,other


In [82]:
# I want lattert o plot which day of the week was the attack so I must create a file with the day or the week in format yyyy-mm-dd
sharks6_activity['date_c'] = pd.to_datetime(sharks6_activity['year_c'].astype(str) + '-' + sharks6_activity['month_c'].astype(str) + '-' + sharks6_activity['day_c'].astype(str), format='%Y-%m-%d', errors='coerce')

C:\Users\psaez\AppData\Local\Temp\ipykernel_19596\4273289699.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks6_activity['date_c'] = pd.to_datetime(sharks6_activity['year_c'].astype(str) + '-' + sharks6_activity['month_c'].astype(str) + '-' + sharks6_activity['day_c'].astype(str), format='%Y-%m-%d', errors='coerce')


In [83]:
sharks6_activity.sample()

,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,year_c,month_c,day_c,Act_class,date_c
2517,Unprovoked,USA,Hawaii,"Honomuni, Moloka'i","Standing in waist-deep water, helping his fath...",Pahu Tanaka,M,10,Right leg bruised & abraded,N,A.M.,"Tiger shark, 2.4 m [8']","J. Borg, p.82",1992.12.28-Tanaka.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,3786.0,1992,12,28,other,1992-12-28


In [84]:
# now I am using the function .dt.day_name() to get the day of the week corresponding to date_c value 
sharks6_activity['day_of_week'] = sharks6_activity['date_c'].dt.day_name()

C:\Users\psaez\AppData\Local\Temp\ipykernel_19596\3821567120.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sharks6_activity['day_of_week'] = sharks6_activity['date_c'].dt.day_name()


In [85]:
final_sharks = sharks6_activity

In [86]:
final_sharks.head()

,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,original order,year_c,month_c,day_c,Act_class,date_c,day_of_week
0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6303.0,2018,06,25,other,2018-06-25,Monday
1,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6302.0,2018,06,18,other,2018-06-18,Monday
2,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6301.0,2018,06,09,surfing,2018-06-09,Saturday
3,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6300.0,2018,06,08,surfing,2018-06-08,Friday
4,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,6299.0,2018,06,04,diving,2018-06-04,Monday


In [87]:
final_sharks['day_of_week'].value_counts()

day_of_week
Sunday       945
Saturday     910
Monday       651
Thursday     645
Wednesday    639
Friday       632
Tuesday      614
Name: count, dtype: int64

In [89]:
final_sharks.to_csv("../data/final_sharks.csv")